In [ ]:
import numpy as np
import pandas as pd
import time
import tensorly as tl
import scipy as sp
import random as rd
import matplotlib.pyplot as plt
from tensorly.decomposition import parafac
from sklearn.metrics import roc_curve, auc,roc_auc_score,precision_recall_curve,average_precision_score,mean_squared_error
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.model_selection import KFold

In [ ]:
tensor_T=np.load("less_Tensor_rdt.npy")

A_t=np.load("t_t_max_min.npy")
A_d=np.load("d_d_symptoms.npy")
A_r=np.load("sim_mir_seq_less.npy")
A_d_1=np.load("mesh_sim.npy")
r_d_tensor=np.load("r_d_tensor.npy")

r_d=np.load("less_r_d_more_matrix_np.npy")
r_t=np.load("less_r_t_more_matrix_np.npy")
t_d=np.load("less_t_d_more_matrix_np.npy")

In [ ]:
pd.read_csv("less_m_t_d_m_d_n.csv")

In [ ]:
pd.read_csv("less_m_t_d_m_r_n.csv")

In [ ]:
pd.read_csv("less_m_t_d_m_t_n.csv")

In [ ]:
def TDMDA(T,S_m,S_t,S_d,r_t,r_d,t_d,r,lamdaU=0.1,lamdaA=0.1,alpha=0.125,beta=0.000001,eta=1,p=1.1,tol=1e-6,max_iter=300):

    dim1,dim2,dim3=T.shape
#     r=int(r)
#     print("r: ",type(r))
    # initialization
    U_1=np.random.random([dim1,r])
    U_2=np.random.random([dim2,r])
    U_3=np.random.random([dim3,r])
    
    Z_1=np.zeros(U_1.shape)
    Z_2=np.zeros(U_2.shape)
    Z_3=np.zeros(U_3.shape)
    
    Y_1=np.zeros(U_1.shape)
    Y_2=np.zeros(U_2.shape)
    Y_3=np.zeros(U_3.shape)
    
    I_1=np.eye(dim1)
    I_2=np.eye(dim2)
    I_3=np.eye(dim3)
    
    D_1 = np.diagflat(S_m.sum(1))
    D_2 = np.diagflat(S_t.sum(1))
    D_3 = np.diagflat(S_d.sum(1))
    
#     DS_1=D_1**(-0.5)
#     DS_2=D_2**(-0.5)
#     DS_3=D_3**(-0.5)
    
    L_1 = D_1 - S_m
    L_2 = D_2 - S_t
    L_3 = D_3 - S_d
    
#     L_1=np.dot(np.dot(DS_1,L_1),DS_1)
#     L_2=np.dot(np.dot(DS_2,L_2),DS_2)
#     L_3=np.dot(np.dot(DS_3,L_3),DS_3)
    
    T_nor=np.linalg.norm(T)
    T_size=dim1*dim2*dim3
    psi_c=T.copy()
    psi_c=1-psi_c
    X=T+psi_c*(T_nor/T_size)
    
    
    
    for i in range(max_iter):
        
        X_old=X.copy()
        eta=p*eta

        Z_1=np.dot(np.linalg.inv(eta*I_1+alpha*L_1+0.00001*I_1),(eta*U_1-Y_1))
        Z_2=np.dot(np.linalg.inv(eta*I_2+alpha*L_2+0.00001*I_2),(eta*U_2-Y_2))
        Z_3=np.dot(np.linalg.inv(eta*I_3+alpha*L_3+0.00001*I_3),(eta*U_3-Y_3))
        
        U_1T = np.dot(U_1.T,U_1)
        U_2T = np.dot(U_2.T,U_2)
        U_3T = np.dot(U_3.T,U_3)
        
        M_X=np.zeros([r,r])
        M_1= CG(M_X, beta*U_1T ,U_3T , beta* np.dot(np.dot(U_1.T,r_d),U_3), lamdaA, 0.01, 200)
        M_2 = CG(M_X, beta*U_1T ,U_2T , beta* np.dot(np.dot(U_1.T,r_t),U_2), lamdaA, 0.01, 200)
        M_3 = CG(M_X, beta*U_2T ,U_3T , beta* np.dot(np.dot(U_2.T,t_d),U_3), lamdaA, 0.01, 200)
#         M_1 =CG(r_t,U_1,U_2,r,beta,lamda,0.0001,20)
#         M_2 = CG(r_d,U_1,U_3,r,beta,lamda,0.0001,20)
#         M_3 = CG(t_d,U_2,U_3,r,beta,lamda,0.0001,20)
        
        
        E_1=tl.tenalg.khatri_rao([U_2, U_3])
        E_2=tl.tenalg.khatri_rao([U_1, U_3])
        E_3=tl.tenalg.khatri_rao([U_1, U_2])
        
        X_1 = tl.unfold(X, 0)
        X_2 = tl.unfold(X, 1)
        X_3 = tl.unfold(X, 2)
        
        XE_1=np.dot(X_1,E_1)
        XE_2=np.dot(X_2,E_2)
        XE_3=np.dot(X_3,E_3)
        
        
        A_1=np.dot(np.dot(r_t,U_2),M_1.T)+np.dot(np.dot(r_d,U_3),M_2.T)
        A_2=np.dot(np.dot(r_t.T,U_1),M_1)+np.dot(np.dot(t_d,U_3),M_3.T)
        A_3=np.dot(np.dot(r_d.T,U_1),M_2)+np.dot(np.dot(t_d.T,U_2),M_3)
        
        ZY_1=eta*Z_1+Y_1
        ZY_2=eta*Z_2+Y_2
        ZY_3=eta*Z_3+Y_3
        
        EE_1=np.dot(E_1.T,E_1)
        EE_2=np.dot(E_2.T,E_2)
        EE_3=np.dot(E_3.T,E_3)
        
        UU_1=np.dot(np.dot(np.dot(M_1,U_2.T),U_2),M_1.T)+np.dot(np.dot(np.dot(M_2,U_3.T),U_3),M_2.T)
        UU_2=np.dot(np.dot(np.dot(M_1.T,U_1.T),U_1),M_1)+np.dot(np.dot(np.dot(M_3,U_3.T),U_3),M_3.T)
        UU_3=np.dot(np.dot(np.dot(M_2.T,U_1.T),U_1),M_2)+np.dot(np.dot(np.dot(M_3.T,U_2.T),U_2),M_3)
        
        U_1=np.dot((XE_1+ beta*A_1+ZY_1),np.linalg.inv(EE_1 + beta*UU_1+lamdaU*np.eye(r)+eta*np.eye(r)))
        U_2=np.dot((XE_2+ beta*A_2+ZY_2), np.linalg.inv(EE_2 + beta*UU_2+lamdaU*np.eye(r)+eta*np.eye(r)))
        U_3=np.dot((XE_3+ beta*A_3+ZY_3), np.linalg.inv(EE_3 + beta*UU_3+lamdaU*np.eye(r)+eta*np.eye(r)))
        
#         E=tl.tenalg.khatri_rao([U_2, U_3])
#         X_cp= tl.fold(np.dot(U_1, E.T), 0, T.shape)
        RE_3=tl.tenalg.khatri_rao([U_1, U_2])
        X_cp=tl.fold(np.dot(U_3,RE_3.T), 2, T.shape)
            
#         print(X)
        X= T+psi_c*X_cp
#         print(X)
        
        Y_1=Y_1+eta*(Z_1-U_1)
        Y_2=Y_2+eta*(Z_2-U_2)
        Y_3=Y_3+eta*(Z_3-U_3)
       
        err=np.linalg.norm(X- X_old)/np.linalg.norm(X_old)
#         print(err)
        
        
        if err < tol:
            print(i," rigerr ",err)
            break
        if err>100:
            print("baderr",err)
            break
    
    RE=tl.tenalg.khatri_rao([U_2, U_3])
    X_pre= tl.fold(np.dot(U_1, RE.T), 0, T.shape)
    return X_pre

In [ ]:
def CG(X_initial, A, B, D, mu, tol, max_iter):

    X = X_initial
    R = D - np.dot(np.dot(A,X),B) - mu * X
    P = np.array(R, copy=True)
#     print("X:",X,"R:",R,"P:",P)

    for i in range(max_iter):
        R_norm = np.trace(np.dot(R,R.T))
#         print("R_norm:",R_norm)
        Q = np.dot(np.dot(A,P),B)+ mu * P
        alpha = R_norm / np.trace(np.dot(Q,P.T))
#         print("alpha:",alpha)
        X = X + alpha * P
        R = R - alpha * Q
        err = np.linalg.norm(R)
        if err < tol:

#             print("CG convergence: iter = %d" % i)
            break
        

        beta = np.trace(np.dot(R,R.T)) / R_norm
        P = R + beta * P
        
    
#     print(X)
    return X

In [ ]:
def forDisMatrixALLDataall(r_d):
    times=10
    for tt in range(times):
        
        print(tt,"****")
        k=5
        y=r_d.copy()
        

        y_true_index =np.argwhere(y==1)
        data_num, _= y_true_index.shape 
        print(data_num)
        index = np.arange(data_num)  # 生成下标  
        np.random.shuffle(index) 
        np.random.seed()
        y_true_index=y_true_index[index] 
        y_un_index= np.argwhere(y == 0)

        kf = KFold(n_splits=k)
        i=0

        for train, test in kf.split(y_true_index):
            print(tt,"-",i,"&&&&&&&")
            y_true_index_test=y_true_index[test]
            y_index=np.append(y_true_index_test,y_un_index,axis=0)
            print(y_index)
            print(len(y_true_index_test))
            print(len(y_index))
            indexname="CVtruerd_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"
            indexname1="CVtrd_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"
            
            np.save(indexname,y_true_index_test)
            np.save(indexname1,y_index)
            i+=1

In [ ]:
forDisMatrixALLDataall(r_d_tensor)

In [ ]:
def forDisMatrixALLData(r_d):
    times=10
    for tt in range(times):
        
        print(tt,"****")
        k=5
        y=r_d.copy()
        

        y_true_index =np.argwhere(y==1)
        data_num, _= y_true_index.shape 
        print(data_num)
        index = np.arange(data_num)  # 生成下标  
        np.random.shuffle(index) 
        np.random.seed()
        y_true_index=y_true_index[index] 
        y_un_index= np.argwhere(y == 0)
      
        for i in range(5):
            indexname="CVtruerd_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"
            y_true_index_test=np.load(indexname)
            y_false_index=np.array(rd.sample(y_un_index.tolist(),len(y_true_index_test)))
            y_index=np.append(y_true_index_test,y_false_index,axis=0)
            print(y_index)
            print(len(y_true_index_test))
            print(len(y_index))
            
        
            indexname1="CV11rd_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"

       
            np.save(indexname1,y_index)
    

In [ ]:
forDisMatrixALLData(r_d_tensor)

In [ ]:
def forDisMatrixALL(r_d,tensor_t,S_r,S_t,S_d,rt,rd,td,r,lamdaU=0.1,lamdaA=0.1,alpha=0.125,beta=0.000001,eta=1,p=1.1):
    
    cnt10=0
   
    times=10
    y_true_list=[]
    y_pre_list=[]
    
    auclist=[]
    auprlist=[]
    
    for tt in range(times):
        
        print(tt,"****")
        k=5
        y_tensor=tensor_t.copy()
        y=r_d.copy()
       
       
        for i in range(k):
            print(tt,"-",i,"&&&&&&&")
            indexname="CV11rd_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"
            
            y_true_index_test=np.load(indexname)
            
            print(len(y_true_index_test))
            
            matrix=y.copy()
            r_d_ad=rd.copy()
            r_d_ad[y_true_index_test[:,0],y_true_index_test[:,1]]=0
            

            tensor=y_tensor.copy()
            tensor[y_true_index_test[:,0],:,y_true_index_test[:,1]]=0
            tensor_rec=TDMDA(tensor,S_r,S_t,S_d,rt,r_d_ad,td,r,lamdaU,lamdaA,alpha,beta,eta,p)
            matrix_rec=np.mean(tensor_rec,axis=1).copy()

            y_true_value=y[y_true_index_test[:,0],y_true_index_test[:,1]]
            y_true=tl.base.tensor_to_vec(y_true_value)
            
           
            
            y_pre_value=matrix_rec[y_true_index_test[:,0],y_true_index_test[:,1]]
            y_pre=tl.base.tensor_to_vec(y_pre_value)
            
               
            y_true_list.extend(y_true.tolist())
            y_pre_list.extend(y_pre.tolist())
           
            
            
            
            fpr,tpr,threshold=roc_curve(y_true,y_pre)
            print(auc(fpr,tpr))
            pre_o,re_o,_threshold=precision_recall_curve(y_true,y_pre)
            print(auc(re_o,pre_o))
            auclist.append(auc(fpr,tpr))
            auprlist.append(auc(re_o,pre_o))
            
    
    return  y_true_list,y_pre_list,np.std(np.array(auclist), ddof = 1),np.std(np.array(auprlist), ddof = 1)

In [ ]:
dim=tensor_T.shape[2]
r=int(dim*0.5)
print(r)
y_true_list,y_pre_list,aucstd,auprstd=forDisMatrixALL(r_d_tensor,tensor_T,A_r,A_t,A_d,r_t,r_d_tensor,t_d,r,0.001,0.001,0.125,0.125,0.1,1.1)

In [ ]:
def forDisMatrixALLa(r_d,tensor_t,S_r,S_t,S_d,rt,rd,td,r,lamdaU=0.1,lamdaA=0.1,alpha=0.125,beta=0.000001,eta=1,p=1.1):
    
    cnt10=0
   
    times=10
    y_true_list=[]
    y_pre_list=[]
    
    auclist=[]
    auprlist=[]
    
    for tt in range(times):
        
        print(tt,"****")
        k=5
        y_tensor=tensor_t.copy()
        y=r_d.copy()
       
       
        for i in range(k):
            print(tt,"-",i,"&&&&&&&")
            indexname="CVtrd_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"
            
            y_true_index_test=np.load(indexname)
            
            print(len(y_true_index_test))
            
            matrix=y.copy()
            r_d_ad=rd.copy()
            r_d_ad[y_true_index_test[:,0],y_true_index_test[:,1]]=0
            
            
        
# #             print(len(y_true_index_test))

            tensor=y_tensor.copy()
            tensor[y_true_index_test[:,0],:,y_true_index_test[:,1]]=0
# #             print(tensor.sum())
            tensor_rec=TDMDA(tensor,S_r,S_t,S_d,rt,r_d_ad,td,r,lamdaU,lamdaA,alpha,beta,eta,p)
            matrix_rec=np.mean(tensor_rec,axis=1).copy()
#             matrixname="matrixRec_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"
#             matrix_rec=np.load(matrixname)
#             print(matrix_rec)
            
            y_true_value=y[y_true_index_test[:,0],y_true_index_test[:,1]]
            y_true=tl.base.tensor_to_vec(y_true_value)
            
           
            
            y_pre_value=matrix_rec[y_true_index_test[:,0],y_true_index_test[:,1]]
            y_pre=tl.base.tensor_to_vec(y_pre_value)
            
               
            y_true_list.extend(y_true.tolist())
            y_pre_list.extend(y_pre.tolist())
           
            
            
#             precision_t,recall_t=get_metrics(np.mat(y_true), np.mat(y_pre))
            
#             precision+=precision_t
#             recall+=recall_t
            
            
            
            fpr,tpr,threshold=roc_curve(y_true,y_pre)
            print(auc(fpr,tpr))
            pre_o,re_o,_threshold=precision_recall_curve(y_true,y_pre)
            print(auc(re_o,pre_o))
            auclist.append(auc(fpr,tpr))
            auprlist.append(auc(re_o,pre_o))
            
       
        
  #     precision/=50
#     recall/=50
#     aupr=auc(recall,precision)
    
    return  y_true_list,y_pre_list,np.std(np.array(auclist), ddof = 1),np.std(np.array(auprlist), ddof = 1)

In [ ]:
dim=tensor_T.shape[2]
r=int(dim*0.5)
print(r)
y_true_lista,y_pre_lista,aucstda,auprstda=forDisMatrixALLa(r_d_tensor,tensor_T,A_r,A_t,A_d,r_t,r_d_tensor,t_d,r,0.001,0.001,0.125,0.125,0.1,1.1)

In [ ]:
def NewDis_Data(tensor):
    times=10
    for tt in range(times):
        
        print(tt,"****")
        k=5
        y_tensor=tensor.copy()
        index_d=y_tensor.shape[2]
        index_d=np.arange(0,index_d,1)
        

    
        data_num= index_d.shape[0]
        index = np.arange(data_num)  # 生成下标 
        np.random.seed()
        np.random.shuffle(index) 
        np.random.seed()
        
        index_d=index_d[index] 
        

       
        kf = KFold(n_splits=k)
        i=0

        for train, test in kf.split(index_d):
            print(tt,"-",i,"&&&&&&&")
            y_true_index_test=index_d[test]
            print(len(y_true_index_test))
            print(y_true_index_test)
            
            indexname="CV_fnewdis_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"

            
            np.save(indexname,y_true_index_test)
            i+=1

In [ ]:
NewDis_Data(tensor_T)

In [ ]:
def NewDis(r_d,tensor_t,S_r,S_t,S_d,rt,rd,td,r,lamdaA,lamdaU,alpha,beta,eta,p):
    y_true_list=[]
    y_pre_list=[]
    times=10
    
    auclist=[]
    auprlist=[]

    for tt in range(times):
        
        print(tt,"****")
        k=5
        
#         mean_tpr5=0.0
#         mean_fpr5=np.linspace(0,1,1000)
#         mean_pre5=0.0
#         mean_re5=np.linspace(0,1,1000)
#         cnt5=0
        y=r_d.copy()
        
       
       
        for i in range(k):
            print(tt,"-",i,"&&&&&&&")
            indexname="CV_fnewdis_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"
            
            y_true_index_test=np.load(indexname)
            print(len(y_true_index_test))
            
            
            tensor=tensor_t.copy()
            rd_ad=rd.copy()
            rd_ad[:,y_true_index_test]=0
            
            tensor[:,:,y_true_index_test]=0
                
            tensor_rec=TDMDA(tensor,S_r,S_t,S_d,rt,rd_ad,td,r,lamdaA,lamdaU,alpha,beta,eta,p)
                
            matrix_rec=np.mean(tensor_rec,axis=1).copy()
            
            y_true_value=y[:,y_true_index_test]
            y_true=tl.base.tensor_to_vec(y_true_value)
            
           
            
            y_pre_value=matrix_rec[:,y_true_index_test]
            y_pre=tl.base.tensor_to_vec(y_pre_value)
            y_true_list.extend(y_true.tolist())
            y_pre_list.extend(y_pre.tolist())
           
            fpr,tpr,threshold=roc_curve(y_true,y_pre)
            print(auc(fpr,tpr))
            
            
            pre_o,re_o,_threshold=precision_recall_curve(y_true,y_pre)
            print(auc(re_o,pre_o))
            
            auclist.append(auc(fpr,tpr))
            auprlist.append(auc(re_o,pre_o))
            
            
    
    return   y_true_list,y_pre_list,np.std(np.array(auclist), ddof = 1),np.std(np.array(auprlist), ddof = 1)

In [ ]:
dim=tensor_T.shape[2]
r=int(dim*0.5)
print(r)
# mean_fpr10,mean_tpr10,mean_auc10,mean_pre10,mean_re10,aupr=NewDis(r_d_tensor,tensor_T,A_r,A_t,A_d,r_t,r_d,t_d,r,0.01,0.001,0.125,0.125,0.1,1.1)
y_true_listn,y_pre_listn,aucstd_n,aucpr_n=NewDis(r_d_tensor,tensor_T,A_r,A_t,A_d,r_t,r_d_tensor,t_d,r,0.001,0.001,0.125,0.125,0.1,1.1)

In [ ]:
fpr,tpr,threshold=roc_curve(y_true_listn,y_pre_listn)
print(auc(fpr,tpr))
pre_o,re_o,_threshold=precision_recall_curve(y_true_listn,y_pre_listn)
print(auc(re_o,pre_o))

In [ ]:
def CV_All_test(tensor_t,S_r,S_t,S_d,rt,rd,td,r,lamdaU=0.1,lamdaA=0.1,alpha=0.125,beta=0.000001,eta=1,p=1.1):
    mean_tpr10=0.0
    mean_fpr10=np.linspace(0,1,2000)
    cnt10=0
    y_true_list=[]
    y_pre_list=[]
    
    auclist=[]
    mselist=[]
#     precision=0
#     recall=0
    times=10
    mse=0
    metrics_tensor = np.zeros((1, 2))
    for tt in range(times):
        
        print(tt,"****")
        k=5
        y=tensor_t.copy()
        
        mean_tpr5=0.0
        mean_fpr5=np.linspace(0,1,3000)
        cnt5=0
       
       
        for i in range(k):
            print(tt,"-",i,"&&&&&&&")
            indexname1="CVtrue_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"
            indexname2="CV11_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"
            y_true_index_test=np.load(indexname2)
            y_true_index_MSE=np.load(indexname1)
            
            y_true_index_test_r_t=np.unique(np.array([y_true_index_MSE[:,0],y_true_index_MSE[:,1]]).T,axis=0)
            y_true_index_test_r_d=np.unique(np.array([y_true_index_MSE[:,0],y_true_index_MSE[:,2]]).T,axis=0)
            y_true_index_test_t_d=np.unique(np.array([y_true_index_MSE[:,1],y_true_index_MSE[:,2]]).T,axis=0)
            
            rt_ad=rt.copy()
            rd_ad=rd.copy()
            td_ad=td.copy()

            rt_ad[y_true_index_test_r_t[:,0],y_true_index_test_r_t[:,1]]=0
            rd_ad[y_true_index_test_r_d[:,0],y_true_index_test_r_d[:,1]]=0
            td_ad[y_true_index_test_t_d[:,0],y_true_index_test_t_d[:,1]]=0
        
#             print(len(y_true_index_test))

            tensor=y.copy()
            tensor[y_true_index_test[:,0],y_true_index_test[:,1],y_true_index_test[:,2]]=0
            tensor_rec=TDMDA(tensor,S_r,S_t,S_d,rt_ad,rd_ad,td_ad,r,lamdaU,lamdaA,alpha,beta,eta,p)
            
            y_true_value=y[y_true_index_test[:,0],y_true_index_test[:,1],y_true_index_test[:,2]]
            y_true=tl.base.tensor_to_vec(y_true_value)
            
            y_simple_true=y[y_true_index_MSE[:,0],y_true_index_MSE[:,1],y_true_index_MSE[:,2]].copy()
            y_true_MSE=tl.base.tensor_to_vec(y_simple_true)
            
            
            y_pre_value=tensor_rec[y_true_index_test[:,0],y_true_index_test[:,1],y_true_index_test[:,2]]
            y_pre=tl.base.tensor_to_vec(y_pre_value)
            
            y_simple_pre= tensor_rec[y_true_index_MSE[:,0],y_true_index_MSE[:,1],y_true_index_MSE[:,2]].copy()
            y_pre_MSE=tl.base.tensor_to_vec(y_simple_pre)
            
            
            y_true_list.extend( y_true.tolist())
            y_pre_list.extend(y_pre.tolist())
            
            mse+=mean_squared_error(y_true_MSE,y_pre_MSE)
#             mse+=mean_squared_error(y_true,y_pre)
            mselist.append(mean_squared_error(y_true_MSE,y_pre_MSE))
            print(mse)
            
            
#             metrics_tensor+=get_metricsTri(np.mat(y_true), np.mat(y_pre))
            
#             precision+=precision_t
#             recall+=recall_t
            
            
            cnt5+=1
            fpr,tpr,threshold=roc_curve(y_true,y_pre)
            auce=auc(fpr,tpr)
            print(auce)
            auclist.append(auce)
            pre_o,re_o,_threshold=precision_recall_curve(y_true,y_pre)
            print(auc(re_o,pre_o))
            mean_tpr5+=np.interp(mean_fpr5,fpr,tpr)
            mean_tpr5[0]=0.0
        
        mean_tpr5/=cnt5
        mean_tpr5[-1]=1.0
            
        cnt10+=1
        mean_tpr10+=np.interp(mean_fpr10,mean_fpr5,mean_tpr5)
        mean_tpr10[0]=0.0
        
    
    mean_tpr10/=cnt10
    mean_tpr10[-1]=1.0
    mean_auc10=auc(mean_fpr10, mean_tpr10)
    mean_pre10,mean_re10,_=precision_recall_curve(y_true_list,y_pre_list)
    aupr=auc(mean_re10, mean_pre10)
# #     precision/=50
# #     recall/=50
# #     aupr=auc(recall,precision)
#     mse/=50
    return  mean_auc10,mse/50,mean_pre10,mean_re10,aupr,np.std(np.array(mselist), ddof = 1),np.std(np.array(auclist), ddof = 1)


In [ ]:
def forMTDDataall(r_d):
    times=10
    for tt in range(times):
        
        print(tt,"****")
        k=5
        y=r_d.copy()
        

        y_true_index =np.argwhere(y==1)
        data_num, _= y_true_index.shape 
        print(data_num)
        index = np.arange(data_num)  # 生成下标  
        np.random.shuffle(index) 
        np.random.seed()
        y_true_index=y_true_index[index] 

        kf = KFold(n_splits=k)
        i=0

        for train, test in kf.split(y_true_index):
            print(tt,"-",i,"&&&&&&&")
            y_true_index_test=y_true_index[test]
            
            print(len(y_true_index_test))
            
            indexname="CVtrue_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"
            np.save(indexname,y_true_index_test)
            i+=1

In [ ]:
forMTDDataall(tensor_T)

In [ ]:
def forMTDData(r_d):
    times=10
    for tt in range(times):
        
        print(tt,"****")
        k=5
        y=r_d.copy()
        

#         y_true_index =np.argwhere(y==1)
#         data_num, _= y_true_index.shape 
#         print(data_num)
#         index = np.arange(data_num)  # 生成下标  
#         np.random.shuffle(index) 
#         np.random.seed()
#         y_true_index=y_true_index[index] 
        y_un_index= np.argwhere(y == 0)
      
        for i in range(5):
            indexname="CVtrue_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"
            y_true_index_test=np.load(indexname)
            y_false_index=np.array(rd.sample(y_un_index.tolist(),len(y_true_index_test)))
            y_index=np.append(y_true_index_test,y_false_index,axis=0)
            print(y_index)
            print(len(y_true_index_test))
            print(len(y_index))
            
        
            indexname1="CV11_"+"%s"%str(tt)+"_"+"%s"%str(i)+".npy"

       
            np.save(indexname1,y_index)
    

In [ ]:
forMTDData(tensor_T)

In [ ]:
dim=tensor_T.shape[2]
r=int(dim*0.3)
print(r)
mean_auctrip,msetrip,mean_pre,mean_re10,aupr,msestd,aucstd=CV_All_test(tensor_T,A_r,A_t,A_d,r_t,r_d,t_d,r,0.001,0.001,0.125,0.125,0.1,1.1)